# Usage of Keras for image_classification


* configure random transformations and normalization operations to be done on your image data during training
* instantiate generators of augmented image batches (and their labels) via .flow(data, labels) or .flow_from_directory(directory). These generators can then be used with the Keras model methods that accept data generators as inputs, fit_generator, evaluate_generator and predict_generator.

In [15]:
from sklearn import datasets
iris = datasets.load_iris()

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

import numpy as np
seed = 13
np.random.seed(seed)

In [13]:
import pandas as pd
from keras.utils import np_utils

In [35]:
Y = np.concatenate((np.array([0]*80),np.array([1]*80), np.array([2]*80), np.array([3]*80), np.array([4]*80)))

In [36]:
#### Create one-hot encoded output

In [37]:
# encode class values as intergers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables ( i.e. one hot encoded)

In [38]:
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y

array([[ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.]])

In [12]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [10]:
def baseline_model():
    #create model
    model = Sequential()
    model.add(Dense(8, input_dim=(100,50)))
    model.add(Activation('relu'))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    #compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [42]:
from keras.preprocessing.image import ImageDataGenerator

In [61]:
batch_size = 10

train_datagen = ImageDataGenerator(
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
rescale=1./255,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')

# only rescaling augmentation for test samples
test_datagen = ImageDataGenerator(rescale=1./255)

In [49]:
train_dir = '../data/processed/train'
test_dir = '../data/processed/test'

In [56]:
# this is a generaotr that will read pictures 
# found in sub folders of 'data/train',
# and indefinitely generate batches of augmented image data
train_generator = train_datagen.flow_from_directory(
                train_dir,
                target_size=(100,50),
                batch_size=batch_size,
                class_mode='categorical') # since we use categoricalentropy loss

Found 400 images belonging to 5 classes.


In [62]:
# this is a similar generatr, for validation/test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(100,50),
    batch_size=batch_size,
    class_mode='categorical')

Found 100 images belonging to 5 classes.


In [132]:
model = Sequential()

model.add(Conv2D(32, 1, input_shape=(100,50,3)))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(1, 1)))

#model.add(Conv2D(32, (1,1)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(1, 1)))

#model.add(Conv2D(64, (1,1)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(1, 1)))


# the model so far outputs 3D feature maps (height, width, features)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [134]:
model.fit_generator(train_generator,
                   steps_per_epoch=2,
                   epochs=10,
                   validation_data=test_generator,
                   validation_steps=2)
model.save_weights('first_try.h5')

Epoch 1/10


ValueError: Error when checking target: expected activation_46 to have 4 dimensions, but got array with shape (16, 5)

In [137]:
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import matplotlib.pyplot as plt

#import pydot

#import graphviz
import numpy as np
import os